# (Deep) Neural Networks

It is the easiest if you work on this Notebook in google Colab where you don't need to install tensorflow on your laptop.

+ Google Colab: https://colab.research.google.com/notebooks/welcome.ipynb#recent=true

+ Go to "upload" to open this notebook in Google Colab.

If you would like to install tensorflow on your local machine, run the following code in a notebok cell: `!pip install tensorflow`


## Neural Net with Fashion MNIST 

Our task is to classify images of clothing into 10 classes  using a neural network model. The goal is to get familiar with neural networks applied on real-world datasets.



In [ ]:
# load required packages
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Activation, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__) #version should be at least 1.15.x

### Dataset
We use the Fashion MNIST dataset by Zalando which contains 70,000 grayscale images each assigned one of 10 clothing categories (e.g., Top, Trouser, Sneaker, ...). The images show individual clothing articles at low resolution (28 by 28 pixels). The dataset is available as tensorflow dataset.

We take the split that comes with the dataset where 60,000 images are used to train the network and 10,000 images to evaluate its prediction performance. 

In [ ]:
# load the training and test data
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# names of class labels (we have ten classes)
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Put the datasets in the necessary shapes (dimensionality) and check this.

In [ ]:
train_images = train_images.reshape((len(train_images),28,28))
test_images = test_images.reshape((len(test_images),28,28))

# check the shapes of the training and test data 
print("shape for training (x) data : ", train_images.shape) # should be: 60'000 Images each with 28x28 pixels
print("shape for training (y) data : ", train_labels.shape)  # 60'000 Labels with 10 classes
print("shape for test (x) data     : ", test_images.shape)  # 10'000 Images with 28x28 pixels
print("shape for test (y) data     : ", test_labels.shape)  # 10'000 Labels with 10 classes

In [ ]:
# to give you an overview of the data plot first 25 images with corresponding labels
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

### Design the Deep Neural Network
The goal here is to deisgn a convolutional neural network for classifying clothing images in ten clothing categories. The first and the last layer of the network are given. It is your choice how deep or complex you want to build your neural network. 

Here are some possible layers you could use:
+ `Flatten()` https://keras.io/api/layers/reshaping_layers/flatten/ 
+ `Dense()` (specify a activation function i.e. use the argument, `activation=...`) https://keras.io/api/layers/core_layers/dense/
+ `Dropout()` https://keras.io/api/layers/regularization_layers/dropout/
+`BatchNormalization` https://keras.io/api/layers/normalization_layers/batch_normalization/
+ `Conv2D()` (specify a activation function i.e. use the argument, `activation=...`)  https://keras.io/api/layers/convolution_layers/convolution2d/
+ `MaxPooling2D()` https://keras.io/api/layers/pooling_layers/max_pooling2d/

*Hint:* Consider that for using the Conv2D model we need to reshape the training and test images to (60000, 28, 28, **1**) and (10'000, 28, 28, **1**), respectively!

In [ ]:
# you can also run this model without adding layers, to see how the most simple model performs
model = Sequential()
#model.add(Flatten(input_shape=(28, 28))) #remove this layer if you want to use Covolutional neural networks

# data reshaped for Convolution2D
train_images=train_images.reshape(60000,28,28,1)
test_images=test_images.reshape(10000,28,28,1)

#### DESIGN YOUR NETWORK HERE ####

#### END YOUR NETWORK DESIGN HERE ####

# final output softmax layer for 10 classes (do not modify this layer)
model.add(Dense(10, activation = 'softmax'))

# print a summary of your model
model.summary()

### Train the model
Compile and train the model you designed. 

You can also adapt the `batch_size`, the number of epochs `epochs` and the optimizer used in the training process.


In [ ]:
# compile model
# you can modify the parameter optimizer with different optimization methods
model.compile(optimizer = 'adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# specify batch size for the optimization (mini-batch)
batch_size = 128
# specify the number of epochs
epochs = 10

# fit model to train data
# we use 10% of the data as validation set
# you can add more 'iterations' by raising the parameter epochs
log = model.fit(train_images, 
                train_labels, 
                batch_size=128,
                epochs=10,
                validation_split=0.1)

# plot accuracy per epoch
plt.plot(log.history['accuracy'], label='Training Accuracy')
plt.plot(log.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()


### Evaluate the model on the test dataset


In [ ]:
# compute the loss and accuracy
test_scores = model.evaluate(test_images, test_labels, verbose=0)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

# you can also make predictions for the test data
predictions = model.predict(test_images)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# get the maximum probability class
max_probability_predictions = np.argmax(predictions, axis=1)
# compute and display the confusion matrix
conf_mat = confusion_matrix(test_labels, max_probability_predictions)
conf_mat = ConfusionMatrixDisplay(conf_mat)

conf_mat.plot()

The cell below visualizes the some example images from the test set along with their true labels and the model predictions. 

In [ ]:
# This function plots the first n predictions with their true label (in brackets) and image
test_images=test_images.reshape(10000,28,28)

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img, cmap=plt.cm.binary)
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# plot prediction and image
num_rows = 7
num_cols = 5
num_images = num_rows*num_cols
plt.figure(figsize=(2*num_cols, 1*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, num_cols, i+1)
  plot_image(i, predictions[i], test_labels, test_images)
plt.tight_layout()
plt.show()